# NLP_Sentiment_Analysis_BTC-ETH

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi import NewsApiClient
import datetime as dt

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jclay\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!

Bad key savefig.frameon in file c:\Users\jclay\anaconda3\envs\pyviz_clone\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.4.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file c:\Users\jclay\anaconda3\envs\pyviz_clone\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.4.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file c:\Users\jclay\anaconda3\envs\p

In [4]:
# Read your api key environment variable
load_dotenv() # 
api_key = os.getenv('NEWS_API_KEY') # create new variable and set equal to a function. os is a module that allows you to interact with the operating system. Then we pass NEWS_API_KEY to the getenv function.
#os.environ['NEWS_API_KEY']
api_key

'7968e07793034c538e9ee10af1bb7b71'

In [5]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key) # create a new variable and set to NewsApiClient. NewsApiClient is a class that allows you to interact with the newsapi.io API. And we pass api_key to the NewsApiClient function.

In [6]:
# Fetch the Bitcoin news articles
articles_btc = newsapi.get_everything(q='bitcoin', language='en', sort_by='relevancy') # q is the search term, language is the language of the articles, sort_by is the sort order.

In [ ]:
# Fetch the Ethereum news articles
articles_eth = newsapi.get_everything(q='ethereum', language='en', sort_by='relevancy')

In [11]:
# Helper function to create the dataframe 
def create_sentiment_df(news):
    articles = []
    for article in news["articles"]:
        try:
            
            """ # This block of code will be executed for every article in the news api 
            # and defines the parts of the article that we want to include in the df """

            title = article["title"]
            description = article["description"]
            date = article["publishedAt"][:10]
            content = article["content"]
            
            # Create the sentiment and define the properties
            # This block of code will be executed for every article in the news api
            # and will define the sentiment of each article and add it to the df
            sentiment = analyzer.polarity_scores(content)
            compound = sentiment["compound"] # compound is the overall sentiment of the article
            pos = sentiment["pos"] # positive sentiment is extracted from the polarity_scores function
            neu = sentiment["neu"] # neutral sentiment is extracted from the polarity_scores function
            neg = sentiment["neg"] # negative sentiment is extracted from polarity_scores scores
            # Append each articles properties and scores in dictionary format to a list
            articles.append({ 
                "title": title, 
                "description": description,
                "date": date,
                "content": content,
                "compound": compound,
                "pos": pos,
                "neu": neu,
                "neg": neg
            })
        except AttributeError: # except AttributeError is a built in error that is raised when an attribute is not found.
            pass # pass is a keyword that tells python to ignore the error and continue with the code.
    
    return pd.DataFrame(articles) # return the dataframe

In [12]:
# Create the Bitcoin sentiment scores DataFrame
df_btc = create_sentiment_df(articles_btc)
df_btc.head()

,title,description,date,content,compound,pos,neu,neg
0,Miami’s Bitcoin Conference Left a Trail of Har...,"For some women, inappropriate conduct from oth...",2022-05-10,"Now, even though there are a number of women-f...",0.0772,0.036,0.964,0.000
1,Why fossil fuel companies see green in Bitcoin...,ExxonMobil and other fossil fuel companies hav...,2022-05-04,A Bitcoin mining site powered by otherwise los...,-0.0516,0.056,0.882,0.061
2,Warren Buffett Says He Wouldn't Take All the B...,Warren Buffett has always been a bitcoin skept...,2022-05-02,Warren Buffett has always been a bitcoin skept...,-0.3269,0.085,0.772,0.143
3,Bitcoin value drops by 50% since November peak,The slide in the value of cryptocurrencies com...,2022-05-09,"Image source, Getty Images\r\nThe value of Bit...",0.3400,0.072,0.928,0.000
4,Luna Foundation Tried to Prop Up Terra’s Crumb...,"As a kid, I remember when my father tried to u...",2022-05-16,"As a kid, I remember when my father tried to u...",0.3818,0.114,0.833,0.052


In [10]:
# Create the Ethereum sentiment scores DataFrame
df_eth = pd.DataFrame(columns=['date', 'title', 'description', 'url', 'sentiment_score'])
df_eth

,date,title,description,url,sentiment_score


In [ ]:
# Describe the Bitcoin Sentiment


In [ ]:
# Describe the Ethereum Sentiment
# YOUR CODE HERE!

### Questions:

Q: Which coin had the highest mean positive score?

A: 

Q: Which coin had the highest compound score?

A: 

Q. Which coin had the highest positive score?

A: 

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
# Instantiate the lemmatizer
# YOUR CODE HERE!

# Create a list of stopwords
# YOUR CODE HERE!

# Expand the default stopwords list if necessary
# YOUR CODE HERE!

In [ ]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Remove the punctuation from text

   
    # Create a tokenized list of the words
    
    
    # Lemmatize words into root words

   
    # Convert the words to lowercase
    
    
    # Remove the stop words
    
    
    return tokens

In [ ]:
# Create a new tokens column for Bitcoin
# YOUR CODE HERE!

In [ ]:
# Create a new tokens column for Ethereum
# YOUR CODE HERE!

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
# YOUR CODE HERE!

In [ ]:
# Generate the Ethereum N-grams where N=2
# YOUR CODE HERE!

In [ ]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
# Use token_count to get the top 10 words for Bitcoin
# YOUR CODE HERE!

In [ ]:
# Use token_count to get the top 10 words for Ethereum
# YOUR CODE HERE!

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [ ]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!

---